In [1]:
import pymongo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string
from unidecode import unidecode
import warnings 
warnings.filterwarnings("ignore")

pd.set_option("display.max_rows", None)

In [2]:
lst_columns = ["Order_details_ID", "Order_ID", "Product_ID", "Promotion_ID", "Order_line_number", "Quantity_ordered", "Unity_price", "Amount_net", "Amount_reduced", "Amount_invoiced"]

In [3]:
# chargement des dataframe dont nous 
df_products = pd.read_csv("dataset/products.csv")
df_promotions = pd.read_csv("dataset/promotions.csv")
df_orders = pd.read_csv("dataset/orders.csv")

In [4]:
df_products.head()

,Product_ID,Category_ID,Product,Product_name,Description_Product,Price,Currency
0,P000001,Cat_0002,9628,Armani Luminous Silk Fond De Teint Eclat Parfait,Teint,30000,FCFA
1,P000002,Cat_0002,16125,Armani Luminous Silk Poudre Fusion Eclat,Teint,33000,FCFA
2,P000003,Cat_0002,5043,Artist By Nocibe Fond De Teint Goutte A Goutte,Teint,12000,FCFA
3,P000004,Cat_0002,10165,Bareminerals Fond De Teint Liquide Tenue Perfo...,Teint,25000,FCFA
4,P000005,Cat_0002,16583,Beauty Concepts Xxl Bronzer Illuminant,Teint,10000,FCFA


In [5]:
def promo_period(df_promotions):
    length_promo = len(df_promotions)
    lst_promo_period = []
    for i in range(length_promo):
        lst_date = []
        start = df_promotions["Promotion_start_date"].iloc[i]
        end =  df_promotions["Promotion_end_date"].iloc[i]
        lst_date.extend([start, end])
        if lst_date not in lst_promo_period:
            lst_promo_period.append(lst_date)
    return lst_promo_period

In [6]:
df_promotions.head(2)

,Promotion_ID,Product_ID,Promo_code,Percentage_reduction,Promotion_start_date,Promotion_end_date
0,Promo_0001,P000002,BeautyYearGift,0.10,2022-11-15,2023-01-20
1,Promo_0002,P000186,BeautyYearGift,0.15,2022-11-15,2023-01-20


In [7]:
def random_product(df_products, df_promotions, lst_promo_perio, lst_product_promo, order_date):
    info_product = []
    product = df_products.sample()
    product_id = product["Product_ID"].iloc[0]
    num_product = product["Product"].iloc[0]
    unit_price = product["Price"].iloc[0]
    promo_period_found = ''
    percentage_reduction = 0
    promo_id = 0
    # vérifions si order_date est incluse dans l'une sous liste
    for promo_period in lst_promo_perio:
        start, end = promo_period
        if start <= order_date <= end :
            promo_period_found = promo_period
            
            # vérifier si product_id à un code promo
            if product_id in lst_product_promo :
                info_promo = df_promotions[ (df_promotions["Promotion_start_date"] == start) & (df_promotions["Promotion_end_date"] == end) & (df_promotions["Product_ID"] == product_id)]
                percentage_reduction = info_promo["Percentage_reduction"].iloc[0]
                promo_code = info_promo["Promo_code"].iloc[0]
                promo_id = info_promo["Promotion_ID"].iloc[0]
            break
 
    info_product.extend([product_id,num_product,unit_price, promo_id, percentage_reduction])
    return info_product

In [8]:
def random_order_detail(df_products, df_promotions, df_orders ):
    count = 0
    length_orders = len(df_orders)
    lst_columns = ["Order_details_ID", "Order_ID", "Product_ID", "Promotion_ID", "Order_line_number", 
                   "Quantity_ordered", "Unity_price", "Amount_net", "Amount_reduced", "Amount_invoiced"]
    lst_promo_perio = promo_period(df_promotions)
    lst_product_promo = (df_promotions["Product_ID"].unique()).tolist()
    lst_order_details = []
    k=0
    while(count < 50) : 
        count = 0
        lst_order_details = []
        for i in range(length_orders):
            
            number_item = df_orders["Number_items"].iloc[i]
            for item in range(1,number_item+1):
                k=k+1
                dic_order_detail = {}
                order_date = df_orders["Order_date"].iloc[i] 
                dic_order_detail["Order_details_ID"] = f"Od{k:06d}"
                dic_order_detail["Order_ID"] = df_orders["Order_ID"].iloc[i]
                info_product = random_product(df_products, df_promotions, lst_promo_perio, lst_product_promo, order_date) # product_id, produit, unit_price, promo_id, percentage_reduction
                dic_order_detail["Product_ID"] =  info_product[0]
                dic_order_detail["Order_line_number"] = item
                dic_order_detail["Quantity_ordered"] = random.choice([1,2,2,3,3,4])
                dic_order_detail["Unity_price"] = info_product[2]
                dic_order_detail["Amount_net"] = info_product[2] * dic_order_detail["Quantity_ordered"]

                if (info_product[3] != 0): # il n'y a pas de promo
                    dic_order_detail["Promotion_ID"] = info_product[3]
                    dic_order_detail["Amount_reduced"] = dic_order_detail["Amount_net"] * info_product[4]
                    dic_order_detail["Amount_invoiced"] = dic_order_detail["Amount_net"] * (1 - info_product[4])

                    count = count +1


                else : 
                    dic_order_detail["Promotion_ID"] = np.nan
                    dic_order_detail["Amount_reduced"] = 0
                    dic_order_detail["Amount_invoiced"] = dic_order_detail["Amount_net"]

                sort_dic_order_details = sorted(dic_order_detail.keys(), key=lambda x: lst_columns.index(x))
                lst_order_details.append(dic_order_detail)
        print(count)
            
    return lst_order_details


In [9]:
lst_order_details = random_order_detail(df_products, df_promotions, df_orders )

32
35
50


In [10]:
len(lst_order_details)

4486

In [11]:
# enregistrement dans un dataset
df_order_details =  pd.DataFrame( columns = lst_columns)
for l_od in lst_order_details :
    df_order_details = df_order_details.append(l_od, ignore_index=True)
    
df_order_details.head()

# enregistrement dans un fichier csv
df_order_details.to_csv("dataset/order_details.csv",encoding='utf-8-sig', index=False)

len(df_order_details)


4486

In [12]:
df_order_details

,Order_details_ID,Order_ID,Product_ID,Promotion_ID,Order_line_number,Quantity_ordered,Unity_price,Amount_net,Amount_reduced,Amount_invoiced
0,Od008973,O000001,P001057,NaN,1,4,21000,84000,0,84000
1,Od008974,O000001,P000603,NaN,2,4,36000,144000,0,144000
2,Od008975,O000001,P000243,NaN,3,3,70000,210000,0,210000
3,Od008976,O000002,P000733,NaN,1,3,15500,46500,0,46500
4,Od008977,O000002,P000117,NaN,2,3,18000,54000,0,54000
5,Od008978,O000002,P001204,NaN,3,2,8000,16000,0,16000
6,Od008979,O000002,P000310,NaN,4,3,70000,210000,0,210000
7,Od008980,O000002,P000921,NaN,5,3,75000,225000,0,225000
8,Od008981,O000003,P000350,NaN,1,4,43000,172000,0,172000
9,Od008982,O000003,P000468,NaN,2,2,32000,64000,0,64000


In [13]:
nombre_lignes_non_nan = df_order_details["Promotion_ID"].dropna().count()
nombre_lignes_non_nan

50

In [14]:
len(df_order_details) #52

4486

In [15]:
df_pro = pd.read_csv("dataset/Promotions.csv")
df_pro

,Promotion_ID,Product_ID,Promo_code,Percentage_reduction,Promotion_start_date,Promotion_end_date
0,Promo_0001,P000002,BeautyYearGift,0.10,2022-11-15,2023-01-20
1,Promo_0002,P000186,BeautyYearGift,0.15,2022-11-15,2023-01-20
2,Promo_0003,P000189,BeautyYearGift,0.12,2022-11-15,2023-01-20
3,Promo_0004,P000004,BeautyYearGift,0.05,2022-11-15,2023-01-20
4,Promo_0005,P000087,BeautyYearGift,0.10,2022-11-15,2023-01-20
5,Promo_0006,P000032,BeautyYearGift,0.07,2022-11-15,2023-01-20
6,Promo_0007,P000031,BeautyYearGift,0.18,2022-11-15,2023-01-20
7,Promo_0008,P000444,BeautyYearGift,0.20,2022-11-15,2023-01-20
8,Promo_0009,P001124,BeautyYearGift,0.05,2022-11-15,2023-01-20
9,Promo_0010,P001126,BeautyYearGift,0.15,2022-11-15,2023-01-20
